# Experiment: presidential elections Ad

Data Collection is (**TODO** explain).  
It is somehow expensive (**TODO** explain).
You can communicate how (**TODO** explain). 

In this tutorial we are going to explain: a), b) , c)

# Table of Contents
* [Experiment: Predict Elections Using Presidential Commercial Campaign](#Experiment:-Predict-Elections-Using-Presidential-Commercial-Campaign)
* [Data collection](#Data-collection)
    * [Set up environment and dependencies](#Set-up-environment-and-dependencies)        
    * [Use pytube to download videos from YouTube](#Use-pytube-to-download-videos-from-YouTube)
    * [Use cv2 to extract frames from video](#Use-cv2-to-extract-frames-from-video)    
    * [Use pydub to chunck videos and convert audio](#Use-pydub-to-chunck-videos-and-convert-audio)
    * [Combine data into a dataframe](#Combine-data-into-a-dataframe)
* [Feature engineering: extract data using Microsoft Azure public cloud services](#Paragraph-3)
    * [Set up containers and upload files (audio & image)](#Set-up-containers-and-upload-files(audio-&-image))
    * [Extract speech from audio using Bing Speech Recognition API](#Extract-speech-from-audio-using-Bing-Speech-Recognition-API)
    * [Extract sentiment and key phrases from text using Text Analytics API](#Extract-sentiment-and-key-phrases-from-text-using-Text-Analytics-API)
    * [Extract images contents and text using Vision API](#Paragraph-3)
    * [Extract facial features using Vision API](#Paragraph-3)
* [Modeling: Classification](#Paragraph-3)
    * [One versus one](#Paragraph-3)
    * [Bag of words](#Paragraph-3)

## Data Collection: prepare data for your experiment

We are going to download videos from the webpage the [Ten of the Most Succesfull Presidential Campaign Ads Ever Made](https://www.kqed.org/lowdown/3955/ten-of-the-best-presidential-campaign-commercials-of-all-time). 
[The Living Room Candidate at Museum of the Moving Image: presidential campaign commercials](http://www.livingroomcandidate.org/)

In [445]:
#import libraries
import os
import pandas as pd
from pytube import YouTube
import requests
import time

In [2]:
#list of URLs in the page (consider writing a script to scrape for greater quantities)
videos_urls = ['https://youtu.be/Y9RAxAgksSE', 
                'https://youtu.be/vs5ORK8RLWk', 
                'https://youtu.be/dDTBnsqxZ3k',
                'https://youtu.be/Qwk_epMblW4', 
                'https://youtu.be/qVcFUIXEDZ8', 
                'https://youtu.be/EU-IBF8nwSY',
                'https://youtu.be/PmwhdDv8VrM', 
                'https://youtu.be/Xq_x3JUwrU0', 
                'https://youtu.be/pbdzMLk9wHQ', 
                'https://youtu.be/jjXyqcx-mYY']

,video_url
0,https://youtu.be/Y9RAxAgksSE
1,https://youtu.be/vs5ORK8RLWk
2,https://youtu.be/dDTBnsqxZ3k
3,https://youtu.be/Qwk_epMblW4
4,https://youtu.be/qVcFUIXEDZ8
5,https://youtu.be/EU-IBF8nwSY
6,https://youtu.be/PmwhdDv8VrM
7,https://youtu.be/Xq_x3JUwrU0
8,https://youtu.be/pbdzMLk9wHQ
9,https://youtu.be/jjXyqcx-mYY


In [3]:
#set directory to the folder where we are going to download the videos
os.chdir('../../data/video/video.mp4/')
dir_video = os.getcwd()

#set directory to folder to download the video thumbnail
os.chdir('../thumbnail/')
dir_thumbnail = os.getcwd()

#print directories where files are going to be saved
print(dir_video)
print(dir_thumbnail)

C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\video.mp4
C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\thumbnail


In [15]:
#set procedure starting time
print('-------------------')
print("Start downloading videos and thumbnails")
print('-------------------')
start = time.time()

#store useful info to lists
titles = []
thumbnail_urls = []
video_file_spec = []

#download videos and thumbnails
for video in videos_urls:

    #instatiate Youtube object passing video URL
    yt = YouTube(video)

    #extract the video's title and thumbnail url
    replacements = {" ": "_", "-": "", "'": "", '"': "", ':':'', '.':'', '(':'', ')':''}
    title = "".join([replacements.get(c, c) for c in yt.title.lower()])
    titles.append(title)
    thumbnail_url = yt.thumbnail_url.lower()
    thumbnail_urls.append(thumbnail_url)

    #choose file to download (filter by extension and resolution and take first in the list)
    video_file = yt.streams.filter(file_extension = 'mp4', res="360p").first()
    video_file_spec.append(video_file)
    
    #download video
    try:
        video_file.download(dir_video, filename=title)
        print('{} download status: success'.format(title))
    except:
        print('{} download status: fail'.format(title))
        
    #set thumbail name and choose path to save it
    thumbnail_name = title+'_'+yt.thumbnail_url.split('/')[-1]
    save_thumbnail_path = (os.path.join(dir_thumbnail, thumbnail_name))
    
    #download thumbnail
    img_data = requests.get(thumbnail_url).content
    with open(save_thumbnail_path, 'wb') as handler:
        try:
            handler.write(img_data)
            print('{} download status: success'.format(thumbnail_name))
        except:
            print('{} download status: fail'.format(thumbnail_name))

#set procedure ending time
end = time.time()
print('-------------------')
print('Download completed')
print('-------------------')
print('It took {} seconds to download 10 Ads and their thumbnails'.format(round(end - start),2))

-------------------
Start downloading videos and thumbnails
-------------------
eisenhower_for_president_1952 download status: success
eisenhower_for_president_1952_default.jpg download status: success
kennedy_for_me_campaign_jingle_jfk_1960 download status: success
kennedy_for_me_campaign_jingle_jfk_1960_default.jpg download status: success
high_quality_famous_daisy_attack_ad_from_1964_presidential_election download status: success
high_quality_famous_daisy_attack_ad_from_1964_presidential_election_default.jpg download status: success
humphrey_laughing_at_spiro_agnew_1968_political_ad download status: success
humphrey_laughing_at_spiro_agnew_1968_political_ad_default.jpg download status: success
mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial download status: success
mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial_default.jpg download status: success
ronald_reagan_tv_ad_its_morning_in_america_again download status: success
ronald_reagan_tv_ad_

In [58]:
#import libraries
import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import make_chunks
#ignore warning

In [9]:
#set path to folder to save audio file
os.chdir('../audio/')
dir_audio = os.getcwd()

print(dir_audio)

C:\Users\popor\iqss_workshop\workshops\public_cloud_computing\data\video\audio


In [63]:
#set procedure starting time
print('-------------------')
print("Start dividing video in chuncks and converting to audio")
print('-------------------')
start = time.time()

#store audio lenght
audio_lenght = []

#convert video to audio, and split in chunks
for title in titles:
    
    #set path to file to convert
    path_to_file = os.path.join(dir_video, title) + '.mp4'
        
    #read file into variable
    myaudio = AudioSegment.from_file(path_to_file, "mp4")
    
    #estimate length
    file_length = round(len(myaudio)/1000, 2)
    audio_lenght.append(file_length)
    
    #Make chunks of fi
    chunk_length_ms = 15000
    chunks = make_chunks(myaudio, chunk_length_ms)

    #take each chunck and export it to path as wav
    dir_file = os.path.join(dir_audio, title)
    for i, chunk in enumerate(chunks):
        chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")
        
    print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(chunks)))
    
#set procedure ending time
end = time.time()
print('-------------------')
print('Process completed')
print('-------------------')
print('It took {} seconds to chunk the video and convert '.format(round(end - start, 2)))

-------------------
Start dividing video in chuncks and converting to audio
-------------------
The video eisenhower_for_president_1952 is long 62.09 seconds and has been split in 5 audio files
The video kennedy_for_me_campaign_jingle_jfk_1960 is long 60.23 seconds and has been split in 5 audio files
The video high_quality_famous_daisy_attack_ad_from_1964_presidential_election is long 66.9 seconds and has been split in 5 audio files
The video humphrey_laughing_at_spiro_agnew_1968_political_ad is long 19.25 seconds and has been split in 2 audio files
The video mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial is long 60.05 seconds and has been split in 5 audio files
The video ronald_reagan_tv_ad_its_morning_in_america_again is long 59.95 seconds and has been split in 4 audio files
The video 1988_george_bush_sr_revolving_door_attack_ad_campaign is long 29.88 seconds and has been split in 2 audio files
The video bill_clinton_hope_ad_1992 is long 60.26 seconds and has be

In [26]:
#TODO split on silence

#set procedure starting time
print('-------------------')
print("Start dividing video in chuncks and converting to audio")
print('-------------------')
start = time.time()

#store audio lenght
audio_lenght = []

#convert video to audio, and plit in chunks
for title in titles:

    #set path to file to convert
    path_to_file = os.path.join(dir_video, title) + '.mp4'
        
    #read file 
    sound = AudioSegment.from_file(path_to_file, "mp4")
    
    #estimate length
    file_length = round(len(sound)/1000, 2)
    audio_lenght.append(file_length)
    

    print(title)
    
    #split audio when it is silent for at least one second or quieter than -16 dBFS
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-16,  keep_silence=200)
    print(len(chunks))
    
    if len(chunks) == 0:
        chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-48,  keep_silence=200)
    print(len(chunks))
    
    if len(chunks) == 1:
        chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-72,  keep_silence=200)
    print(len(chunks))
    
    # now recombine the chunks so that the parts are at least 10 sec long
    target_length = 10 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length start a new one
            output_chunks.append(chunk)
    
    print(file_length,len(chunks))
    
    dir_file = os.path.join(dir_audio, title)
    #take each chunck and save it as wav
    for i, chunk in enumerate(output_chunks):
        chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")
        
    print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(output_chunks)))
    
#set procedure ending time
end = time.time()
print('-------------------')
print('Process completed')
print('-------------------')
print('It took {} seconds to chunk the video and convert '.format(round(end - start, 2)))

-------------------
Start dividing video in chuncks and converting to audio
-------------------
eisenhower_for_president_1952
5
5
5
62.09 5
The video eisenhower_for_president_1952 is long 62.09 seconds and has been split in 4 audio files
kennedy_for_me_campaign_jingle_jfk_1960
12
12
12
60.23 12
The video kennedy_for_me_campaign_jingle_jfk_1960 is long 60.23 seconds and has been split in 4 audio files
high_quality_famous_daisy_attack_ad_from_1964_presidential_election
0
0
0


IndexError: list index out of range

In [56]:
#sample code to check split on silence

title_HS = titles[4]
path_to_file = os.path.join(dir_video, title_HS) + '.mp4'
sound = AudioSegment.from_file(path_to_file, "mp4")
file_length = round(len(sound)/1000, 2)

chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-16,  keep_silence=200) #,  keep_silence=200
print(title_HS, len(chunks))

if len(chunks) == 0:
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-32,  keep_silence=200)
print(len(chunks))

if len(chunks) == 0:
    chunks = split_on_silence(sound, min_silence_len=300, silence_thresh=-32,  keep_silence=200)
print(len(chunks))

target_length =  10 * 1000
    
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length start a new one
        output_chunks.append(chunk)
        
print(file_length,len(chunks))

dir_file = os.path.join(dir_audio, title_HS)
#take each chunck and save it as wav
for i, chunk in enumerate(output_chunks):
    chunk.export(dir_file + "_chunck_{}.wav".format(i+1), format="wav")

print('The video {0} is long {1} seconds and has been split in {2} audio files'.format(title, file_length,len(output_chunks)))

mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial 15
15
15
60.05 15
The video high_quality_famous_daisy_attack_ad_from_1964_presidential_election is long 60.05 seconds and has been split in 1 audio files


| title | min_silence_len| silence_thresh |keep_silence| target_length |
|---|---|---|---|---|
| eisenhower | 500 | -16 | 200 | 10 |
| kennedy | 500 | -16 | 200 | 10 |
| LBJ | 500 | -32 | 200 | 5 |
| HHH | 500 | -16 | 200 | 5 |



In [65]:
#add some predictors
year = ['1952','1960','1964','1968','1972','1984','1988','1992','2004','2008']
candidate = ['IKE', 'JFK', 'LBJ', 'HHH', 'NIX','---','---','---','---','---' ]
party = ['republican','democratic','democratic','republican', 'republican','---','---','---','---','---',]
win = ['-','-','-',0,'-','-','-','-','-','-']

df = pd.DataFrame({'video_url':videos_urls,
                   'thumbnail_url':thumbnail_urls,
                   'titles':titles,
                   'length[sec]':audio_lenght,
                   'year':year,
                   'candidate_name':candidate,
                   'party': party,
                   'win': win})

df

,video_url,thumbnail_url,titles,length[sec],year,candidate_name,party,win
0,https://youtu.be/Y9RAxAgksSE,https://i.ytimg.com/vi/y9raxagksse/default.jpg,eisenhower_for_president_1952,62.09,1952,IKE,republican,-
1,https://youtu.be/vs5ORK8RLWk,https://i.ytimg.com/vi/vs5ork8rlwk/default.jpg,kennedy_for_me_campaign_jingle_jfk_1960,60.23,1960,JFK,democratic,-
2,https://youtu.be/dDTBnsqxZ3k,https://i.ytimg.com/vi/ddtbnsqxz3k/default.jpg,high_quality_famous_daisy_attack_ad_from_1964_...,66.90,1964,LBJ,democratic,-
3,https://youtu.be/Qwk_epMblW4,https://i.ytimg.com/vi/qwk_epmblw4/default.jpg,humphrey_laughing_at_spiro_agnew_1968_politica...,19.25,1968,HHH,republican,0
4,https://youtu.be/qVcFUIXEDZ8,https://i.ytimg.com/vi/qvcfuixedz8/default.jpg,mcgovern_defense_plan_ad_nixon_1972_presidenti...,60.05,1972,NIX,republican,-
5,https://youtu.be/EU-IBF8nwSY,https://i.ytimg.com/vi/eu-ibf8nwsy/default.jpg,ronald_reagan_tv_ad_its_morning_in_america_again,59.95,1984,---,---,-
6,https://youtu.be/PmwhdDv8VrM,https://i.ytimg.com/vi/pmwhddv8vrm/default.jpg,1988_george_bush_sr_revolving_door_attack_ad_c...,29.88,1988,---,---,-
7,https://youtu.be/Xq_x3JUwrU0,https://i.ytimg.com/vi/xq_x3juwru0/default.jpg,bill_clinton_hope_ad_1992,60.26,1992,---,---,-
8,https://youtu.be/pbdzMLk9wHQ,https://i.ytimg.com/vi/pbdzmlk9whq/default.jpg,historical_campaign_ad_windsurfing_bushcheney_04,30.09,2004,---,---,-
9,https://youtu.be/jjXyqcx-mYY,https://i.ytimg.com/vi/jjxyqcx-myy/default.jpg,yes_we_can__barack_obama_music_video,270.21,2008,---,---,-


In [66]:
#dump the dataframe on a file
import pickle
import json

#dump the dataframe on a file
df.to_pickle('../Ads_presidential_election')
#load dataframe on a file
#df = pd.read_pickle('../Ads_presidential_election')

In [ ]:
#TODO

#other features
###number of words

#advanced features
###extract frames from video (https://stackoverflow.com/questions/27481993/extracting-image-from-video-at-a-given-time-using-opencv)
###number of views
###users comments on you tube

## Feature Extraction

### Set up containers and upload files (audio & image)

In [149]:
def make_public_container(storage_name, storage_key, container_name):
    """"create blob service and blob container. return blob service"""

    blob_service = BlockBlobService(account_name= storage_name, account_key=storage_key)
    container_name = container_name
    new_container = blob_service.create_container(container_name, public_access=PublicAccess.Container) 
    #blob_service.set_container_acl(new_container_name, public_access=PublicAccess.Container)

    if new_container == True:
        print('{} container succesfully created'.format(ncontainer_name))
    else:
        print('{] something went wrong: check parameters and subscription'.format(container_name))
        
    return blob_service

In [421]:
def get_files(dir_files):
    """"store file name, extension and path """
    
    files_name = []
    files_path = []
    files_extension = []
    
    for root, directories, files in os.walk(dir_files):
        for file in files:
            files_name.append(file)
            files_path.append(os.path.join(root,file))
            files_extension.append(file.split('.')[-1])
            
    return files_name, files_path, files_extension

In [424]:
def upload_file(blob_service, container, file, path, extension, content_type):
        #TODO: CREATE BLOB SERVICE
    
        #upload blob using the arguments: container, file path, file name, and content type
        try:
            blob_service.create_blob_from_path(container, file, path, content_settings=ContentSettings(content_type= content_type+extension))    
            print("{} BLOB upload status: successful".format(file))
        
        except:
            print("{} BLOB upload status: failed".format(file))

In [402]:
import pickle
with open('C:/Users/popor/Desktop/azure_services_keys_v1.1.json', 'rb') as handle:
    azure_keys = pickle.load(handle)
    
azure_keys['STORAGE']

{'NAME': 'cloudcomputingplayground',
 'API_KEY': 'ItdsjxJIAo0DGyShdBBiQotdEZ+u4SgMhTNQtDtFAY3rupviiam0k/EyxwOA+taDwAaJcNGKADpEjAJN3fBLww=='}

In [433]:
##################
#RUN IF NECESSARY#
##################

#delete container
#delete_container = blob_service.delete_container(NEW_CONTAINER_NAME)
#print("{} delition status success: {}".format(NEW_CONTAINER_NAME, delete_container))

myaudiofiles delition status success: True


In [436]:
#import libraries
from azure.storage.blob import BlockBlobService, PublicAccess
from azure.storage.blob import ContentSettings
import os

#set directory to audio files
if os.getcwd().split('\\')[-1] == 'audio':
    dir_files = os.getcwd()
else:
    os.chdir('../../data/video/audio/')
    dir_files = os.getcwd()
    
#find names and paths
files_name, files_path, files_extension = get_files(dir_files)

#set storage name and keys
STORAGE_NAME = azure_keys['STORAGE']['NAME']
STORAGE_KEY = azure_keys['STORAGE']['API_KEY']

#set a name for a new container
NEW_CONTAINER_NAME ='myaudio'

In [437]:
#make container function (exist already, test it)

blob_service = BlockBlobService(account_name= STORAGE_NAME, account_key=STORAGE_KEY)
new_container = blob_service.create_container(NEW_CONTAINER_NAME) 
blob_service.set_container_acl(NEW_CONTAINER_NAME, public_access=PublicAccess.Container)
print("{} BLOB container has been successfully created: {}".format(NEW_CONTAINER_NAME, new_container))

myaudio BLOB container has been successfully created: True


In [440]:
#set uploading procedure starting time
print('-------------------')
print("Start uploading files")
print('-------------------')
start = time.time()

#set content type of the file, in this case is a audio .wav
content_type = 'audio/x-'

#upload all files at once to the new container
for path, file, ext in zip(files_path, files_name, files_extension):
    upload_file(blob_service, NEW_CONTAINER_NAME, file, path, ext, content_type)
    
    #add print only failed otherwise good to go
    
#set procedure ending time
end = time.time()
print('-------------------')
print('Uploading completed')
print('-------------------')
print('It took {} seconds to upload the files'.format(round(end - start, 2)))

1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_1.wav BLOB upload status: successful
1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_2.wav BLOB upload status: successful
1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_3.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_1.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_2.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_3.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_4.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_5.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_6.wav BLOB upload status: successful
bill_clinton_hope_ad_1992_chunck_7.wav BLOB upload status: successful
eisenhower_for_president_1952_chunck_1.wav BLOB upload status: successful
eisenhower_for_president_1952_chunck_2.wav BLOB upload status: successful
eisenhower_for_president_1952_chunck_3.wav BLOB upload status: succe

## Extract text from audio using Bing Speech Recognition API

In [446]:
#import libraries
import pandas as pd

import requests
import urllib
import uuid
import json

In [443]:
#get list of blob (function)
uploaded_file = blob_service.list_blobs(NEW_CONTAINER_NAME)
blob_name_list = []
for blob in uploaded_file:
    blob_name_list.append(blob.name)

In [450]:
#configure API access
SPEECH_RECOGNITION_KEY = azure_keys['SPEECH_RECOGNITION']['API_KEY']
uri_token_speech = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'
url_speech = 'https://speech.platform.bing.com/recognize'

In [452]:
#set procedure starting time
print('-------------------')
print("Start speech to text conversion")
print('-------------------')
start = time.time()

#store http response and json file
responses = []
http_responses = []

#run speech recognition on uploaded audio files (i.e. extension .wax)
for blob_name in blob_name_list:
    if blob_name.split('.')[-1] == 'wav':

        #request for token
        uri_token = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'
        headers = {'Content-Length': '0', #need this?
                   'Ocp-Apim-Subscription-Key': SPEECH_RECOGNITION_KEY}
        api_response = requests.post(uri_token_speech, headers=headers)
        access_token = str(api_response.content.decode('utf-8'))

        #convert blob to bytes
        blob = blob_service.get_blob_to_bytes(NEW_CONTAINER_NAME, blob_name)

        #request for speech recognition service
        headers = {'Authorization': 'Bearer {0}'.format(access_token),
                   'Content-type': 'audio/wav', 
                   'codec': 'audio/pcm', 
                   'samplerate': '16000'}
        params = urllib.parse.urlencode({'scenarios': 'ulm',
                                         'appid': 'D4D52672-91D7-4C74-8AD8-42B1D98141A5',
                                         'locale': 'en-US',
                                         'device.os': 'PC',
                                         'version': '3.0',
                                         'format': 'json',
                                         'instanceid': str(uuid.uuid1()),
                                         'requestid': str(uuid.uuid1())})
        api_response = requests.post(url_speech, headers=headers, params=params, data=blob.content)
        print('{} had a {} response'.format(blob_name, api_response))

        #extract data from response
        res_json = json.loads(api_response.content.decode('utf-8'))
        http_responses.append(api_response)
        responses.append(res_json)

#load output next cell       
        
#set procedure ending time
end = time.time()
print('-------------------')
print('Conversion completed')
print('-------------------')
print('It took {} seconds to chunk the video and convert '.format(round(end - start, 2)))

-------------------
Start speech to text conversion
-------------------
1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_1.wav had a <Response [200]> response
1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_2.wav had a <Response [200]> response
1988_george_bush_sr_revolving_door_attack_ad_campaign_chunck_3.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_1.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_2.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_3.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_4.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_5.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_6.wav had a <Response [200]> response
bill_clinton_hope_ad_1992_chunck_7.wav had a <Response [200]> response
eisenhower_for_president_1952_chunck_1.wav had a <Response [200]> response
eisenhower_for_president_1952_chunck_2.wav had a <Response 

In [584]:
#organize response output
status = []
name = []
lexical = []
request_id = []
confidence = []

#select variables from output
for i, response in enumerate(responses):
    if responses[i]['header']['status'] == 'success':
        status.append(responses[i]['header']['status'])
        name.append(responses[i]['header']['name'])
        lexical.append(responses[i]['header']['lexical'])
        request_id.append(responses[i]['header']['properties']['requestid'])
        confidence.append(responses[i]['results'][0]['confidence'])
    else:
        status.append('Error')
        name.append('Nan')
        lexical.append('Nan')
        request_id.append('Nan')
        confidence.append('Nan')

#combine output into df
df_log_response = pd.DataFrame({'file_name' : blob_name_list,
                                'stt_http_response' :  http_responses,
                                'stt_id' : request_id,
                                'stt_status' : status,
                                'stt_name' : name,
                                'stt_text' : lexical,
                                'stt_confidence' : confidence})

#display df
df_log_response.head()

## dump to pickle

,file_name,stt_http_response,stt_id,stt_status,stt_name,stt_text,stt_confidence
0,1988_george_bush_sr_revolving_door_attack_ad_c...,<Response [200]>,49e7ae57-2d45-423a-afeb-b9e405d28ffa,success,who is governor michael dukakis vitov mandator...,who is governor michael dukakis vitov mandator...,0.8767573
1,1988_george_bush_sr_revolving_door_attack_ad_c...,<Response [200]>,8e327848-32ca-4d66-abd7-636b2c1019f9,success,impolicy gave weekend frontales to first degre...,impolicy gave weekend frontales to first degre...,0.8108339
2,1988_george_bush_sr_revolving_door_attack_ad_c...,<Response [200]>,a9a576e0-4a40-4b84-bb02-188a13cd9baf,success,large how michael dukakis says he wants to do ...,large how michael dukakis says he wants to do ...,0.8396592
3,bill_clinton_hope_ad_1992_chunck_1.wav,<Response [200]>,4fa5c0c3-4c8b-4988-91b3-34e0028915bc,success,I was born a little town called hope arkansas ...,i was born a little town called hope arkansas ...,0.8724898
4,bill_clinton_hope_ad_1992_chunck_2.wav,<Response [200]>,7de8dcc0-b7e3-4ec4-923d-e7f43e8de2d0,success,very limited income it was in 1963 that I went...,very limited income it was in nineteen sixty t...,0.8587455


In [522]:
#recompone text from speech recognition service into a df 
dict_speech_recognition = dict()
candidate_president = ['eisenhower',
                       'george_bush_sr',
                       '1964',
                       'humphrey',
                       'jfk',
                       'nixon',
                       'ronald_reagan',
                       'bill_clinton',
                       'bushcheney',
                       'barack_obama'] 

#extract text for each candidate and join it
for name in candidate_president:    
    dict_name = dict()
    audio_text = []

    for i, entry in enumerate(df_log_response.loc[:,'file_name']):
        if name in entry:
            #uncomment the line below and indent the next
            #if you want to get a ride of Nan
            #if df_re.loc[i, 'tts_text'] != 'Nan': 
            audio_text.append(df_log_response.loc[i, 'stt_text'])
                        
    n_words = []
    for words in audio_text: 
        n_words.append(int(len(words.split(' '))))
    words_count = sum(n_words)    
    
    joined_audio = " ".join(audio_text)
    dict_name['stt_text'] = joined_audio
    dict_name['stt_words_count'] = words_count
    dict_speech_recognition[name] = dict_name 

In [592]:
#convert dictionary to df
df_stt = pd.DataFrame.from_dict(dict_speech_recognition , orient='index').reset_index()
df_stt.columns =  'title', 'stt_text', 'stt_words_count'
df_stt

,title,stt_text,stt_words_count
0,1964,Nan play hello by standing ben Nan please are ...,50
1,barack_obama,Nan how Nan Nan how what Nan what people of th...,33
2,bill_clinton,i was born a little town called hope arkansas ...,173
3,bushcheney,i'm george W bush and i approve this message i...,78
4,eisenhower,i for president for president i like my comput...,31
5,george_bush_sr,who is governor michael dukakis vitov mandator...,63
6,humphrey,Nan Nan,2
7,jfk,Nan do you wanna man for president who season ...,17
8,nixon,the mcgovern defense plan he would cut the mar...,84
9,ronald_reagan,it's morning again in america today more men a...,112


In [628]:
#output of the text from the audio of a selected Ad
print('Below, the text used in the presidential campaign Ad by candidate {}:\n"{}"'.format(df_stt.loc[2,'title'], df_stt.loc[2,'stt_text'].replace('Nan', '').replace('   ', ' ')))

Below, the text used in the presidential campaign Ad by candidate bill_clinton:
"i was born a little town called hope arkansas three months after my father died i remembered old two story house where i live in the grandparents very limited income it was in nineteen sixty three that i went to washington invent president kennedy at the boys nation program an IRA member just thinking morning kredible country this was it somebody like me you had no money or anything would be given the opportunity to meet the president rocket really do public service 'cause i care so much about people i work my way through law school with part time jobs anything i could find after i graduated i really didn't care about making a lot of money i just wanna go home and see if i can make a difference between work hard and education and healthcare create jobs and we real progress now it's exhilarating tomato think that is present i could help to change all are peoples lives for the better and bring hope back to t

### Extract sentiment and key phrases from text using Text Analytics API

In [541]:
#configure API access
TEXT_ANALYTICS_KEY = azure_keys['TEXT_ANALYTICS']['API_KEY']
uri_sentiment = 'https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/sentiment'
uri_key_phrases = 'https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases'

#set REST headers
headers = {}
headers['Ocp-Apim-Subscription-Key'] = TEXT_ANALYTICS_KEY
headers['Content-Type'] = 'application/json'
headers['Accept'] = 'application/json'

In [618]:
#set procedure starting time
print('-------------------')
print("Start text analysis")
print('-------------------')
start = time.time()

#store text analysis to list
sentiment_text = []
key_phrases = []
sentiment_mean_key_phrases = []   

#perform on text for each audio
for i, entry in enumerate(df_stt.index):
    text = df_stt.loc[i,'stt_text'].replace('Nan', '').replace('   ', ' ')

    #create request to determine sentiment from text
    data = json.dumps({"documents":[{"id":str(uuid.uuid1()), "language":"en", "text":text}]}).encode('utf-8')
    request = urllib.request.Request(uri_sentiment, data, headers)
    response = urllib.request.urlopen(request)
    responsejson = json.loads(response.read().decode('utf-8'))
    try:
        sentiment = responsejson['documents'][0]['score']
    except:
        sentiment = 'Nan'
    sentiment_text.append(sentiment)

    #create request to determine key phrases from text
    data = data
    request = urllib.request.Request(uri_key_phrases, data, headers)
    response = urllib.request.urlopen(request)
    responsejson = json.loads(response.read().decode('utf-8'))
    try:
        key_phrase = responsejson['documents'][0]['keyPhrases']
    except:
        key_phrase = 'Nan'
    key_phrases.append(key_phrase)
    
    #create request to determine sentiment from key phrases
    sentiment_key_phrases = []
    for key in key_phrase:
        data = json.dumps({"documents":[{"id":str(uuid.uuid1()), "language":"en", "text":key}]}).encode('utf-8')
        request = urllib.request.Request(uri_sentiment, data, headers)
        response = urllib.request.urlopen(request)
        responsejson = json.loads(response.read().decode('utf-8'))
        sentiment = responsejson['documents'][0]['score']
        sentiment_key_phrases.append(round(sentiment, 2))
        time.sleep(1)

    sentiment_mean = sum(sentiment_key_phrases)/len(sentiment_key_phrases)
    sentiment_mean_key_phrases.append(sentiment_mean)

#assign new column to df_stt
df_stt['ta_sentiment_text'] = sentiment_text
df_stt['ta_key_phrases'] = key_phrases
df_stt['ta_sentiment_key_phrases'] = sentiment_mean_key_phrases

#set procedure ending time
end = time.time()
print('-------------------')
print('Text analysis completed')
print('-------------------')
print('It took {} to perform text analysis'.format(round(end - start, 2)))

-------------------
Start text analysis
-------------------
-------------------
Text analysis completed
-------------------
It took 144.0 to perform text analysis


In [630]:
#display dataset after text analytics
df_text_analytics = df_stt.copy()
df_text_analytics

,title,stt_text,stt_words_count,sentiment_text,key_phrases,sentiment_key_phrases
0,1964,Nan play hello by standing ben Nan please are ...,50,0.856453,"[stakes, god, president johnson, s children, v...",0.762500
1,barack_obama,Nan how Nan Nan how what Nan what people of th...,33,0.243644,"[people, nation false hope]",0.815000
2,bill_clinton,i was born a little town called hope arkansas ...,173,0.5,"[cause i, present i, graduated i, lot of money...",0.571613
3,bushcheney,i'm george W bush and i approve this message i...,78,0.5,"[john kerry lead carey, john kerry whichever w...",0.677273
4,eisenhower,i for president for president i like my comput...,31,0.918349,"[president i, computer billy graham washingto...",0.700000
5,george_bush_sr,who is governor michael dukakis vitov mandator...,63,0.0557907,"[massachusetts america, governor michael dukak...",0.529167
6,humphrey,Nan Nan,2,Nan,Nan,0.400000
7,jfk,Nan do you wanna man for president who season ...,17,0.5,"[wanna man, president]",0.380000
8,nixon,the mcgovern defense plan he would cut the mar...,84,0.5,"[cut navy personnel, navy fleet, mcgovern defe...",0.602667
9,ronald_reagan,it's morning again in america today more men a...,112,0.5,"[short years, half, young men, rates, leadersh...",0.590588


In [660]:
#load data collection df and merge with Text analytics
df_data_collection = pd.read_pickle('../Ads_presidential_election')

#make column title look the same to join dataset
df_data_collection['titles'] = df_data_collection['titles'].apply(
    lambda value: 'eisenhower' if value == 'eisenhower_for_president_1952'
    else 'jfk' if value == 'kennedy_for_me_campaign_jingle_jfk_1960'  
    else '1964' if value == 'high_quality_famous_daisy_attack_ad_from_1964_presidential_election'
    else 'humphrey' if value == 'humphrey_laughing_at_spiro_agnew_1968_political_ad'
    else 'nixon' if value == 'mcgovern_defense_plan_ad_nixon_1972_presidential_campaign_commercial'
    else 'ronald_reagan' if value == 'ronald_reagan_tv_ad_its_morning_in_america_again'
    else 'george_bush_sr' if value == '1988_george_bush_sr_revolving_door_attack_ad_campaign'
    else 'bill_clinton' if value == 'bill_clinton_hope_ad_1992'
    else 'bushcheney' if value == 'historical_campaign_ad_windsurfing_bushcheney_04'
    else 'barack_obama' if value == 'yes_we_can__barack_obama_music_video'
    else 'unknown')

#make a copy and display the dataset
df_data_collection = df_data_collection.iloc[:,2:].copy()
df_data_collection

,titles,length[sec],year,candidate_name,party,win
0,eisenhower,62.09,1952,IKE,republican,-
1,jfk,60.23,1960,JFK,democratic,-
2,1964,66.90,1964,LBJ,democratic,-
3,humphrey,19.25,1968,HHH,republican,0
4,nixon,60.05,1972,NIX,republican,-
5,ronald_reagan,59.95,1984,---,---,-
6,george_bush_sr,29.88,1988,---,---,-
7,bill_clinton,60.26,1992,---,---,-
8,bushcheney,30.09,2004,---,---,-
9,barack_obama,270.21,2008,---,---,-


In [659]:
#combine two df
df_data_collection.set_index('titles')
df_text_analytics.set_index('title')
df_experiment = pd.concat([df_data_collection, df_text_analytics], axis=1)

#display dataset for experiment
df_experiment

,titles,length[sec],year,candidate_name,party,win,title,stt_text,stt_words_count,sentiment_text,key_phrases,sentiment_key_phrases
0,eisenhower,62.09,1952,IKE,republican,-,1964,Nan play hello by standing ben Nan please are ...,50,0.856453,"[stakes, god, president johnson, s children, v...",0.762500
1,jfk,60.23,1960,JFK,democratic,-,barack_obama,Nan how Nan Nan how what Nan what people of th...,33,0.243644,"[people, nation false hope]",0.815000
2,1964,66.90,1964,LBJ,democratic,-,bill_clinton,i was born a little town called hope arkansas ...,173,0.5,"[cause i, present i, graduated i, lot of money...",0.571613
3,humphrey,19.25,1968,HHH,republican,0,bushcheney,i'm george W bush and i approve this message i...,78,0.5,"[john kerry lead carey, john kerry whichever w...",0.677273
4,nixon,60.05,1972,NIX,republican,-,eisenhower,i for president for president i like my comput...,31,0.918349,"[president i, computer billy graham washingto...",0.700000
5,ronald_reagan,59.95,1984,---,---,-,george_bush_sr,who is governor michael dukakis vitov mandator...,63,0.0557907,"[massachusetts america, governor michael dukak...",0.529167
6,george_bush_sr,29.88,1988,---,---,-,humphrey,Nan Nan,2,Nan,Nan,0.400000
7,bill_clinton,60.26,1992,---,---,-,jfk,Nan do you wanna man for president who season ...,17,0.5,"[wanna man, president]",0.380000
8,bushcheney,30.09,2004,---,---,-,nixon,the mcgovern defense plan he would cut the mar...,84,0.5,"[cut navy personnel, navy fleet, mcgovern defe...",0.602667
9,barack_obama,270.21,2008,---,---,-,ronald_reagan,it's morning again in america today more men a...,112,0.5,"[short years, half, young men, rates, leadersh...",0.590588


# CLEAN JUNK DOWN HERE

In [582]:
text_list = df_stt['key_phrases'][0]

In [611]:
#store sentiment mean for each list of key phrasestext analysis to list
sentiment_mean_key_phrases = []   


#perform on key phrases for each text
for i, entry in enumerate(df_stt.index):
    key_phrase = df_stt['key_phrases'][i]
    

    #create request to determine sentiment from key phrases

    sentiment_key_phrases = []

    for key in key_phrase:
        data = json.dumps({"documents":[{"id":str(uuid.uuid1()), "language":"en", "text":key}]}).encode('utf-8')
        request = urllib.request.Request(uri_sentiment, data, headers)
        response = urllib.request.urlopen(request)
        responsejson = json.loads(response.read().decode('utf-8'))
        sentiment = responsejson['documents'][0]['score']
        sentiment_key_phrases.append(round(sentiment, 2))
        time.sleep(1)

    sentiment_mean = sum(sentiment_key_phrases)/len(sentiment_key_phrases)
    sentiment_mean_key_phrases.append(sentiment_mean)

df_stt['sentiment_key_phrases'] = sentiment_mean_key_phrases

In [612]:
df_stt

,title,stt_text,stt_words_count,sentiment_text,key_phrases,sentiment_key_phrases
0,1964,Nan play hello by standing ben Nan please are ...,50,0.856453,"[stakes, god, president johnson, s children, v...",0.762500
1,barack_obama,Nan how Nan Nan how what Nan what people of th...,33,0.243644,"[people, nation false hope]",0.815000
2,bill_clinton,i was born a little town called hope arkansas ...,173,0.5,"[cause i, present i, graduated i, lot of money...",0.571613
3,bushcheney,i'm george W bush and i approve this message i...,78,0.5,"[john kerry lead carey, john kerry whichever w...",0.677273
4,eisenhower,i for president for president i like my comput...,31,0.918349,"[president i, computer billy graham washingto...",0.700000
5,george_bush_sr,who is governor michael dukakis vitov mandator...,63,0.0557907,"[massachusetts america, governor michael dukak...",0.529167
6,humphrey,Nan Nan,2,Nan,Nan,0.400000
7,jfk,Nan do you wanna man for president who season ...,17,0.5,"[wanna man, president]",0.380000
8,nixon,the mcgovern defense plan he would cut the mar...,84,0.5,"[cut navy personnel, navy fleet, mcgovern defe...",0.602667
9,ronald_reagan,it's morning again in america today more men a...,112,0.5,"[short years, half, young men, rates, leadersh...",0.590588


In [583]:
sentiment_score = []

for text in text_list:

    #create request to determine sentiment from text
    data = json.dumps({"documents":[{"id":str(uuid.uuid1()), "language":"en", "text":key_phrase}]}).encode('utf-8')
    request = urllib.request.Request(uri_sentiment, data, headers)
    response = urllib.request.urlopen(request)
    responsejson = json.loads(response.read().decode('utf-8'))
    sentiment = responsejson['documents'][0]['score']
    sentiment_score.append(round(sentiment, 2))
    
score = sum(sentiment_score)/len(sentiment_score)
score

#add to df_stt and to df from DC

0.7649999999999999

In [ ]:
#create request to determine sentiment from key phrases
sentiment_mean_key_phrases = []   
sentiment_key_phrases = []

for key in key_phrase:
    data = json.dumps({"documents":[{"id":str(uuid.uuid1()), "language":"en", "text":key}]}).encode('utf-8')
    request = urllib.request.Request(uri_sentiment, data, headers)
    response = urllib.request.urlopen(request)
    responsejson = json.loads(response.read().decode('utf-8'))
    sentiment = responsejson['documents'][0]['score']
    sentiment_key_phrases.append(round(sentiment, 2))
    
sentiment_mean = sum(sentiment_key_phrases)/len(sentiment_key_phrases)
sentiment_mean_key_phrases.append(sentiment_mean)

df_stt['sentiment_key_phrases'] = sentiment_mean_key_phrases

In [170]:
azure_storage_account_name = 'cloudcomputingplayground' # add name
azure_storage_account_key = azure_keys['STORAGE']['API_KEY']
blob_container_name = 'cloudcomputingcontainer'
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

In [171]:
blob_name = 'Eisenhower_1952.chunk2.wav'
blob = blob_service.get_blob_to_bytes(blob_container_name, blob_name)

In [172]:
uri_token = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'

headers = {'Content-Length': '0', 
           'Ocp-Apim-Subscription-Key': speech_key}

api_response = requests.post(uri_token, headers=headers)

access_token = str(api_response.content.decode('utf-8'))

In [136]:
blob.content[0:20]

b'RIFFD((\x00WAVEfmt \x10\x00\x00\x00'

In [173]:
url_stt_api = 'https://speech.platform.bing.com/recognize' # service address 

headers = {
           'Authorization': 'Bearer {0}'.format(access_token),
           'Content-type': 'audio/wav', 'codec': 'audio/pcm', 'samplerate': '16000'}

params = urllib.parse.urlencode({
    'scenarios': 'ulm',
    'appid': 'D4D52672-91D7-4C74-8AD8-42B1D98141A5', # dont change, it is fixed by design
    'locale': 'en-US', # speech in english
    'device.os': 'PC',
    'version': '3.0',
    'format': 'json', # return value in json
    'instanceid': str(uuid.uuid1()), # any guid
    'requestid': str(uuid.uuid1()),
})

api_response2 = requests.post(url_stt_api, headers=headers, params=params, data=blob.content)

In [174]:
api_response2

<Response [200]>

In [178]:
res_json2 = json.loads(api_response2.content.decode('utf-8'))
#text2 = res_json2['results'][0]['lexical']
res_json2

{'version': '3.0',
 'header': {'status': 'error',
  'properties': {'requestid': 'dc0538ff-1755-4f44-8b4b-d94511da15a8',
   'FALSERECO': '1'}}}

In [122]:
blob_service = BlockBlobService(account_name= storage_name, account_key=storage_key)
uploaded_file = blob_service.list_blobs(container_name)
blob_url_format = 'https://{0}.blob.core.windows.net/{1}/{2}'

blob_bytes = blob_service.get_blob_to_bytes(container_name, blob_name)

token = get_token(speech_key)
r = get_text(token, blob_bytes)

In [123]:
r

<Response [403]>

In [118]:
result = []
for blob in uploaded_file:
    print(blob.name)
    blob_url = blob_url_format.format(blob_service.account_name, container_name, blob.name)
    print(blob_url)
    blob_bytes = blob_service.get_blob_to_bytes(container_name, str(blob.name))
    print(blob_bytes.content[0:20])
    r = get_text(token, blob_bytes)
    print(r)
    #result.append(r)

eisenhower_for_president_1952_chunck_1.wav
https://cloudcomputingplayground.blob.core.windows.net/audio/eisenhower_for_president_1952_chunck_1.wav
b'RIFF\xc4\xea\x1a\x00WAVEfmt \x10\x00\x00\x00'
<Response [403]>


In [114]:
result

[<Response [403]>]

In [186]:
#speech recognition (basic words count / bag of words)

def get_token(speech_key):
    """return an authorization token by making a 
    HTTP POST request to Cognitive Services with a valid API key"""
    
    url = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'
    headers = {'Ocp-Apim-Subscription-Key': speech_key}
    
    token = requests.post(uri_token, headers=headers)

    return(token)

In [103]:
def get_text(token, blob):
    """Request that the Bing Speech API convert the audio to text"""

    url =  'https://speech.platform.bing.com/recognize'
    
    headers = {'Authorization': 'Bearer {0}'.format(token),
               'Content-type': 'audio/wav; codec=audio/pcm; samplerate=16000'}
    
    #'Accept': 'application/json',
    
    params = urllib.parse.urlencode({'scenarios': 'ulm',
                                     'appid': 'D4D52672-91D7-4C74-8AD8-42B1D98141A5',
                                     'locale': 'en-US',
                                     'device.os': 'PC',
                                     'version': '3.0',
                                     'format': 'json',
                                     'instanceid': str(uuid.uuid1()),
                                     'requestid': str(uuid.uuid1())})
    
    response = requests.post(url, headers=headers, params=params, data=blob.content)
    return(response)
#     r = requests.post(url, headers=headers, data=stream_audio_file(audio))
#     results = json.loads(r.content)
#     return(results)

In [ ]:
def stream_audio_file(speech_file, chunk_size=1024):
    # Chunk audio file
    with open(speech_file, 'rb') as f:
        while 1:
            data = f.read(1024)
            if not data:
                break
            yield data

In [ ]:
YOUR_AUDIO_FILE = 'https://cloudcomputingplayground.blob.core.windows.net/cloudcomputingcontainer/Eisenhower_1952.chunk0.wav'


In [ ]:



#key phrases
#sentiment analyses
#assemble into a dataframe


#detect face in thumbnails
#analyze image
#character recognition
#add to dataframe

In [ ]:
from IPython.display import Audio, display
import requests
import urllib
import uuid

In [ ]:
#import library and retrieve keys

#from azure library import methods to use storage 
from azure.storage.blob import BlockBlobService, PublicAccess

#retrive your keys
import pickle
with open('guides/keys/azure_services_keys.json', 'rb') as handle:
    azure_keys = pickle.load(handle)

#select storage account name and API key from keys
storage_name = azure_keys['STORAGE']['NAME']
storage_key = azure_keys['STORAGE']['API_KEY']

In [ ]:
# create a new container

#blob service
blob_service = BlockBlobService(storage_name, storage_key)

In [ ]:
#set a name for a new container
new_container_name ='videocontainer'

In [ ]:
#create a new container and set public access
try:
    container_status = blob_service.create_container(new_container_name, public_access=PublicAccess.Container) 
    print("{} creation success status: {}".format(new_container_name, container_status))
except:
    print("{} creation failed".format(new_container_name))

In [ ]:
os.getcwd()+'\\data\\video\\video.wav\\'

In [ ]:
blob_service.create_blob_from_path(new_container_name, local_file_name, full_path)

In [ ]:
#get the file from local 
local_path = os.getcwd()+'\\data\\video\\video.wav'
local_file_name = "Eisenhower_1952.chunk0.wav"
full_path = os.path.join(local_path, local_file_name)
full_path


# Upload the created file
try:
    blob_service.create_blob_from_path(new_container_name, local_file_name, full_path)    
    print("{} upload status: successful".format(local_file_name))
except:
    print("{} upload status: failed".format(local_file_name))

In [ ]:
def retrive_blob_list(keys, container_name):
    """ 
    function to get a list of blobs' URLs
    INPUT: - dictionary with storage info. dictionary format: {storage:{storage_name:name,storage_api_key:api_key}}
           - container name      
    OUTPUT: a list of BLoBs' name and URL
    """
    storage_name = keys['STORAGE']['NAME']
    storage_key = keys['STORAGE']['API_KEY']
    blob_service = BlockBlobService(storage_name, storage_key)
    uploaded_file = blob_service.list_blobs(container_name)
    blob_url_format = 'https://{0}.blob.core.windows.net/{1}/{2}'
    #store blobs' name and URLs in list
    blob_name_list = []
    blob_url_list = []
    # retrive each blob name
    for blob in uploaded_file:
        blob_name_list.append(blob.name)
        blob_url_list.append(blob_url_format.format(blob_service.account_name, container_name, blob.name))
    return blob_name_list, blob_url_list

In [ ]:
#get a complete list of images' name and urls
retrive_blob_list(azure_keys,new_container_name)

In [ ]:
# load speech file to process
blob_name = 'Eisenhower_1952.chunk1.wav'
blob = blob_service.get_blob_to_bytes(new_container_name, blob_name)

wav_bytes = Audio(data=blob.content)
display(wav_bytes)

In [ ]:
url_token_api = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'

In [ ]:
headers = {'Content-Length': '0', 'Ocp-Apim-Subscription-Key': storage_key}

#"Content-Type", "application/octet-stream"

api_response = requests.post(url_token_api, headers=headers)

In [ ]:
api_response

In [ ]:
access_token = str(api_response.content.decode('utf-8'))

# Service
# Call Speech to text service
url_stt_api = 'https://speech.platform.bing.com/recognize' # service address 

headers = {'Authorization': 'Bearer {0}'.format(access_token), 
           'Content-type': 'audio/wav', 
           'codec': 'audio/pcm',  
           'samplerate': '16000'}

# 'Content-Length': len(blob.content), \

params = urllib.parse.urlencode({
    'scenarios': 'ulm',
    'appid': 'D4D52672-91D7-4C74-8AD8-42B1D98141A5',
    'locale': 'en-US',
    'device.os': 'PC',
    'version': '3.0',
    'format': 'json',
    'instanceid': str(uuid.uuid1()),
    'requestid': str(uuid.uuid1()),
})

api_response = requests.post(url_stt_api, headers=headers, params=params, data=blob.content)

In [ ]:
api_response

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

In [555]:
#import library to display notebook as HTML
import os
from IPython.core.display import HTML

#path to .ccs style script
cur_path = os.path.dirname(os.path.abspath("__file__"))
new_path = os.path.relpath('..\\..\\..\\styles\\custom_styles_public_cloud_computing.css', cur_path)

#function to display notebook
def css():
    style = open(new_path, "r").read()
    return HTML(style)

In [556]:
#run this cell to apply HTML style
css()